# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="Tennis.app")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [4]:
env.brains['TennisBrain']

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [6]:
# for i in range(1, 6):                                      # play game for 5 episodes
#     env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
#     states = env_info.vector_observations                  # get the current state (for each agent)
#     scores = np.zeros(num_agents)                          # initialize the score (for each agent)
#     while True:
#         actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
#         actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
#         env_info = env.step(actions)[brain_name]           # send all actions to tne environment
#         next_states = env_info.vector_observations         # get next state (for each agent)
#         rewards = env_info.rewards                         # get reward (for each agent)
#         dones = env_info.local_done                        # see if episode finished
#         scores += env_info.rewards                         # update the score (for each agent)
#         states = next_states                               # roll over states to next time step
#         if np.any(dones):
#             print("Episode finished")# exit loop if episode finished
# #             break
#     print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

In [7]:
from models import MLPModel
from agent import Agent
from collector import Collector
from trainers import PPOTrainer

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [9]:
model_config = {
    "input_size": 24,
    "num_actions": 2,
    "activation": "relu",

    "sigma0": .5,

    "hidden_sizes": (32, 32, 32, 32),
}

agents = {
    "Agent0": Agent(MLPModel(model_config)),
    "Agent1": Agent(MLPModel(model_config))
}

In [10]:
trainer_config = {
    "steps": 500,

    # Tensorboard settings
    "tensorboard_name": "overnight",  # str, set explicitly
    
    "gamma": .99,  # Discount factor
    "tau": .95,

    # PPO
    "ppo_config": {
        # GD settings
        "optimizer": "adam",
        "optimizer_kwargs": {
            "lr": 1e-3,
            "betas": (0.9, 0.999),
            "eps": 1e-7,
            "weight_decay": 0,
            "amsgrad": False
        },


        # "batch_size": 64,
        "minibatches": 10,

        # PPO settings
        "ppo_steps": 5,
        "eps": 0.1,  # PPO clip parameter
        "target_kl": 0.01,  # KL divergence limit
        "value_loss_coeff": 0.1,

        "entropy_coeff": 0.0,

        "max_grad_norm": 0.5,

        # GPU
        "use_gpu": False,
    }
}

trainer = PPOTrainer(agents, env, trainer_config)

In [11]:
trainer.train(4000, trainer.path)

  0%|          | 0/4000 [00:00<?, ?it/s]

Begin training, logged in /Users/ariel/drlnd_logs/overnight_2020-05-07_18-26-42


100%|██████████| 4000/4000 [1:48:31<00:00,  1.63s/it]


In [13]:
import torch

In [14]:
collector = Collector(agents, env)

In [15]:
weights0 = torch.load("agents/Agent0/weights_3999")
weights1 = torch.load("agents/Agent1/weights_3999")

In [16]:
collector.update_agent_state_dict({"Agent0": weights0, "Agent1": weights1})

In [17]:
data = collector.collect_episodes(100, gamma=1, tau=1, disable_tqdm=False)

100%|██████████| 100/100 [02:20<00:00,  1.40s/it]


In [21]:
from utils import get_episode_rewards

In [23]:
ep_rewards = get_episode_rewards(data)

In [31]:
print(f"Mean reward across 100 episodes: {ep_rewards.mean():.3f}")
print("The reward is higher than the 0.5 threshold, so the environment has been solved successfully!")

Mean reward across 100 episodes: 1.004
The reward is higher than the 0.5 threshold, so the environment has been solved successfully!


When finished, you can close the environment.

In [32]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```